<a href="https://colab.research.google.com/github/nvjob/colab.google/blob/main/nb/piper_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colab Anti-Disconnect

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>

In [3]:
# Check GPU type

!nvidia-smi

Sat Mar 22 13:29:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Google drive connect

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Install software

!git clone -q https://github.com/rhasspy/piper
%cd /content/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy>=1.19.0 onnxruntime>=1.11.0 pytorch-lightning==1.7.0 torch==1.11.0
!pip install -q torchtext==0.12.0 torchvision==0.12.0
!pip install -q torchaudio==0.11.0 torchmetrics==0.11.4
!bash build_monotonic_align.sh
!apt-get install -q espeak-ng
!gdown -q "1EWEb7amo1rgFGpBFfRD4BKX3pkjVK1I-" -O "/content/piper/src/python/patch.zip"
!unzip -o -q "patch.zip"
%cd /content

/content/piper/src/python
Requested pytorch-lightning==1.7.0 from https://files.pythonhosted.org/packages/2f/07/57fd85991eec4311b9945369158bca1322f801686d701322a65789852d46/pytorch_lightning-1.7.0-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    torch (>=1.9.*)
           ~~~~~~^
Please use pip<24.1 if you need to use this version.
ERROR: Could not find a version that satisfies the requirement pytorch-lightning==1.7.0 (from versions: 0.0.2, 0.2, 0.2.2, 0.2.3, 0.2.4, 0.2.4.1, 0.2.5, 0.2.5.1, 0.2.5.2, 0.2.6, 0.3, 0.3.1, 0.3.2, 0.3.3, 0.3.4, 0.3.4.1, 0.3.5, 0.3.6, 0.3.6.1, 0.3.6.3, 0.3.6.4, 0.3.6.5, 0.3.6.6, 0.3.6.7, 0.3.6.8, 0.3.6.9, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.4.8, 0.4.9, 0.5.0, 0.5.1, 0.5.1.2, 0.5.1.3, 0.5.2, 0.5.2.1, 0.5.3, 0.5.3.1, 0.5.3.2, 0.5.3.3, 0.6.0, 0.7.1, 0.7.3, 0.7.5, 0.7.6, 0.8.1, 0.8.3, 0.8.4, 0.8.5, 0.9.0, 0.10.0, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.0.8, 1.1.0, 1.1.1, 1.1.

In [5]:
# Configuration
config = {
    "language": "ru",
    "single_speaker": True,
    "dataset": "/content/drive/MyDrive/dataset",
    "output": "/content/output",
    "resume_from_checkpoint": "/content/drive/MyDrive/irina_baba.ckpt",
    "quality": "medium",
    "max_epochs": 5000,
    "batch_size": 20,
    "checkpoint_epochs": 100,
    "max_phoneme_ids": 800,
    "resample": False,
    "sample_rate": "22050",
    "validation_split": 0.0,
    "num_test_examples": 0,
    "dataset_format": "ljspeech",
    "precision": 32
}

In [9]:
# Preprocess dataset

import os

if not os.path.exists(config['output']):
  os.makedirs(config['output'])

force_sp = " --single-speaker" if config['single_speaker'] else ""

%cd /content/piper/src/python

if config['resample']:
  !python resample.py --input_dir "/content/dataset/wavs" --output_dir "/content/dataset/wavs_resampled" --output_sr {config['sample_rate']} --file_ext "wav"
  !mv /content/dataset/wavs_resampled/* /content/dataset/wavs

!python -m piper_train.preprocess \
  --language {config['language']} \
  --input-dir {config['dataset']} \
  --output-dir {config['output']} \
  --dataset-format {config['dataset_format']} \
  --sample-rate {config['sample_rate']} \
  --max-workers 4 \
  {force_sp}


/content/piper/src/python
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/piper/src/python/piper_train/preprocess.py", line 17, in <module>
    from piper_phonemize import (
ModuleNotFoundError: No module named 'piper_phonemize'


In [ ]:
# Train

get_ipython().system(f'''
python -m piper_train \
--dataset-dir {config['output']} \
--accelerator 'gpu' \
--devices 1 \
--batch-size {config['batch_size']} \
--validation-split {config['validation_split']} \
--num-test-examples {config['num_test_examples']} \
--quality "{config['quality']}" \
--checkpoint-epochs {config['checkpoint_epochs']} \
--max_epochs {config['max_epochs']} \
--precision {config['precision']} \
--max-phoneme-ids {config['max_phoneme_ids']} \
--resume_from_checkpoint "{config['resume_from_checkpoint']}"
''')

In [ ]:
# Export model to ONNX

import os
import shutil

# Find latest checkpoint
checkpoint_dir = os.path.join(config['output'], "lightning_logs", "version_0", "checkpoints")
checkpoints = [os.path.join(checkpoint_dir, f) for f in os.listdir(checkpoint_dir) if f.endswith(".ckpt")]
latest_checkpoint = max(checkpoints, key=os.path.getctime)

# Export path
export_path = os.path.join(config['output'], "model.onnx")

!python -m piper_train.export_onnx {latest_checkpoint} {export_path}
shutil.copy(
    os.path.join(config['output'], "config.json"),
    f"{export_path}.json"
)

print(f"Model exported to {export_path}")